In [1]:
from gensim import utils, corpora, models, similarities
from simserver import SessionServer
import pandas as pd

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
server = SessionServer('/Users/lindsay/Documents/Data Science/Insight/Project/simserver')

In [3]:
# load the current corpus
local_corpus = corpora.MmCorpus('gensim/corpus_tfidf.mm')
print(local_corpus)

MmCorpus(5804 documents, 51857 features, 4368165 non-zero entries)


In [4]:
# load pre-processed text
preproc_text = pd.read_pickle('gensim/preprocessed_text_v2.pkl')

In [5]:
corpus = [{'id' : int(x['id']),
           'tokens' : x['text']}
         for ind, x in preproc_text.iterrows()]

In [6]:
# upload corpus to server
utils.upload_chunked(server, corpus, chunksize=1000)

In [7]:
# train server (uses LSI and default dimensionality of 400 over a TF-IDF representation)
server.train(corpus, method='lsi')

ERROR:sqlitedict:failed to delete /var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldict76d91f
Traceback (most recent call last):
  File "//anaconda/lib/python2.7/site-packages/sqlitedict.py", line 283, in terminate
    os.remove(self.filename)
OSError: [Errno 2] No such file or directory: '/var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldict76d91f'
ERROR:sqlitedict:failed to delete /var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldict7e2bc8
Traceback (most recent call last):
  File "//anaconda/lib/python2.7/site-packages/sqlitedict.py", line 283, in terminate
    os.remove(self.filename)
OSError: [Errno 2] No such file or directory: '/var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldict7e2bc8'
ERROR:sqlitedict:failed to delete /var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldict366a8d
Traceback (most recent call last):
  File "//anaconda/lib/python2.7/site-packages/sqlitedict.py", line 283, in terminate
    os.remove(self.filename)
OSError: [Errno 2] No such file or di

In [8]:
# index documents we trained on
server.index(corpus)

ERROR:sqlitedict:failed to delete /var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldict92d516
Traceback (most recent call last):
  File "//anaconda/lib/python2.7/site-packages/sqlitedict.py", line 283, in terminate
    os.remove(self.filename)
OSError: [Errno 2] No such file or directory: '/var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldict92d516'
ERROR:sqlitedict:failed to delete /var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldictb087e8
Traceback (most recent call last):
  File "//anaconda/lib/python2.7/site-packages/sqlitedict.py", line 283, in terminate
    os.remove(self.filename)
OSError: [Errno 2] No such file or directory: '/var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldictb087e8'
ERROR:sqlitedict:failed to delete /var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldictd0298d
Traceback (most recent call last):
  File "//anaconda/lib/python2.7/site-packages/sqlitedict.py", line 283, in terminate
    os.remove(self.filename)
OSError: [Errno 2] No such file or di

In [15]:
# query for similar documents
sim_result = server.find_similar('18183', max_results=100)

In [26]:
# query for text not in corpus
# for this to work, need to create a pre-processing function that
# matches the pre-processing that occurs for the corpus
doc = {'tokens': [unicode('wrestl')]}
server.find_similar(doc)

[('18183', 0.9643458127975464, None),
 ('12691', 0.9349740743637085, None),
 ('16392', 0.8801846504211426, None),
 ('18414', 0.8541254997253418, None),
 ('17351', 0.8292737007141113, None),
 ('15734', 0.820961594581604, None),
 ('18412', 0.778027355670929, None),
 ('13911', 0.761350154876709, None),
 ('14913', 0.7594928741455078, None),
 ('18413', 0.7543889880180359, None),
 ('14625', 0.753095269203186, None),
 ('16449', 0.7153584957122803, None),
 ('16766', 0.6918432116508484, None),
 ('16789', 0.6909932494163513, None),
 ('12600', 0.6831110119819641, None),
 ('17528', 0.6799274682998657, None),
 ('17246', 0.6555585861206055, None),
 ('13309', 0.6014439463615417, None),
 ('15052', 0.5402518510818481, None),
 ('17787', 0.5177507996559143, None),
 ('17778', 0.48597252368927, None),
 ('12897', 0.4845631420612335, None),
 ('14891', 0.47384870052337646, None),
 ('18257', 0.4428785741329193, None),
 ('13440', 0.44049760699272156, None),
 ('17949', 0.38341179490089417, None),
 ('12534', 0.38